In [1]:

import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import imageio
import cupy as cp
from tqdm import tqdm

In [2]:
def get_Hamiltonian2D(x, dt=1, dx=1, m=1, hdot=1, potential = 'none'):
    '''
    returns H - Hamiltonian for lhs and H_rhs - Hamiltonian for rhs parts of Crank-Nicolson method
    '''
    
    dj = 1+1j*dt*hdot/(2*m*dx**2) #elenemts on the main diagonal of H
    oj = -1j*dt*hdot/(4*m*dx**2) #elements on the upper and lower diagonal of H
    diagonal = cp.ones(len(x), dtype=complex)*dj
    off_diagonal = cp.ones(len(x)-1,dtype = complex)*oj
    
    H = cp.diag(diagonal) + cp.diag(off_diagonal, 1) + cp.diag(off_diagonal, -1)
    
    dj_rhs = 1-1j*dt*hdot/(2*m*dx**2) #elenemts on the main diagonal of H_rhs
    oj_rhs = 1j*dt*hdot/(4*m*dx**2) #elements on the upper and lower diagonal of H_rhs
    diagonal_rhs = cp.ones(len(x), dtype=complex)*dj
    off_diagonal_rhs = cp.ones(len(x)-1,dtype = complex)*oj
    H_rhs = cp.diag(diagonal_rhs) + cp.diag(off_diagonal_rhs, 1) + cp.diag(off_diagonal_rhs, -1)
    
    n = cp.shape(H)[0]
    I = cp.identity(n,dtype = complex)
    
    H_2d = cp.kron(H,I) + cp.kron(I,H)
    H_2drhs = cp.kron(H_rhs,I) + cp.kron(I,H_rhs)
    
    if potential == 'none':
        return H_2d, H_2drhs

In [3]:
def gaus2d(x,y,sigma=1,mu=0,p0=0,h=1):
    '''
    initial conditions at t=0 as Gaussian wave packet
    '''
    psi = 1/(cp.pi**(1/4)*cp.sqrt(sigma))*cp.exp(-((x-mu)**2 + (y-mu)**2)/(2*sigma**2))*cp.exp(1j*p0*x/h)
    return psi

In [4]:
hdot = 1
w = 1
m = 1
dt = 0.5
dx = 0.29
x = cp.arange(-10,10,dx)

In [5]:
gaussa_2d = cp.empty((len(x),len(x)),dtype=complex)
for i, xi in enumerate(x):
    for j, xj in enumerate(x):
        gaussa_2d[i,j] = gaus2d(xi,xj)

In [6]:
H2d,H2d_rhs = get_Hamiltonian2D(x)

In [7]:
H2d.shape

(4761, 4761)

In [8]:
initial_cond2d=gaussa_2d.flatten()

b = cp.dot(H2d_rhs,initial_cond2d)

In [9]:
time = 50
res = cp.zeros((time,len(x)**2), dtype = complex)
res[0,:] = cp.linalg.solve(H2d, b)

for i in tqdm(range(1, time)):
    res[i,:] = cp.linalg.solve(H2d, res[i-1,:])

100%|██████████████████████████████████████████████████████████████████████████████████| 49/49 [09:54<00:00, 12.13s/it]


In [12]:
def pic(M,name):
    fig, ax = plt.subplots(figsize=(10,8))
    ax.imshow(np.abs(M.get().reshape(len(x),len(x)))**2)
    plt.axis('off')
    fig.savefig(name)
    plt.close(fig);

In [13]:
plt.ioff()
files = list()
for i in range(50):
    M = res[i,:]
    pic(M,f'{i+1000}animation.png')
    files.append(f'{i+1000}animation.png')
    

In [14]:
images = list()
for file in files:
    images.append(imageio.imread(file))
imageio.mimsave(f'shr.gif', images, fps = 17);

C:\Users\ivang\AppData\Local\Temp\ipykernel_5264\1071562279.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file))
